In [48]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import signal

import csv
import numpy as np
from numpy import array
import random
from scipy import signal
import matplotlib.pyplot as plt

import json
import subprocess

In [15]:
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [2]:
def read_tsv_to_ndarray(path):
    narrays_list = []
    values = []

    with open(path) as tsvfile:
      reader = csv.reader(tsvfile, delimiter='\t')
      for row in reader:
        values = row[1:]  
    
        values = [float(i) for i in values]  
        
        a = array( values )
        
        narrays_list.append(a)
    
    return array(narrays_list)

In [3]:
part_file_path = '/home/user/time_series_compression_project/datasets/UCRArchive_2018/StarLightCurves/StarLightCurves'

train_file_path = part_file_path + '_TRAIN.tsv'
test_file_path = part_file_path + '_TEST.tsv'

ts_train = read_tsv_to_ndarray(train_file_path)
ts_test = read_tsv_to_ndarray(test_file_path)

all_ts = np.append(ts_train, ts_test, axis=0)

random.shuffle(all_ts)
random.shuffle(all_ts)
random.shuffle(all_ts)

print("Shape of all_data: {0}".format(all_ts.shape))

Shape of all_data: (9236, 1024)


In [4]:
all_ts_reshaped = np.reshape(all_ts, (4*9236, 256), order='F')

print(all_ts_reshaped.shape)

(36944, 256)


# Read template json

In [51]:
filename = '/home/user/projects/tsimulus_ts_generate/generator_configuration.json'
with open(filename, 'r') as f:
    datastore = json.load(f)

datastore["generators"][1]["points"]

{'00:00:00.000': -2.5,
 '01:00:00.000': 4.49,
 '02:00:00.000': 3.32,
 '03:00:00.000': 3.94,
 '04:00:00.000': 5.36,
 '05:00:00.000': 4.72,
 '06:00:00.000': 4.74,
 '07:00:00.000': 4.23,
 '08:00:00.000': 3.87,
 '09:00:00.000': 1.76,
 '10:00:00.000': 0.78,
 '11:00:00.000': -2.15,
 '12:00:00.000': -3.46,
 '13:00:00.000': -3.65,
 '14:00:00.000': -4.06,
 '15:00:00.000': -5.76,
 '16:00:00.000': -4.73,
 '17:00:00.000': -5.88,
 '18:00:00.000': -4.56,
 '19:00:00.000': -3.28,
 '20:00:00.000': -2.78,
 '21:00:00.000': -0.76,
 '22:00:00.000': 0.41,
 '23:00:00.000': 0.95}

# Generate time intervals, step = 5.4 min, 256 values

In [52]:
dts = [dt.strftime('%H:%M:%S.000') for dt in 
       datetime_range(datetime(2016, 9, 1, 0), datetime(2016, 9, 1, 23), 
       timedelta(minutes=5.4))]

dts

['00:00:00.000',
 '00:05:24.000',
 '00:10:48.000',
 '00:16:12.000',
 '00:21:36.000',
 '00:27:00.000',
 '00:32:24.000',
 '00:37:48.000',
 '00:43:12.000',
 '00:48:36.000',
 '00:54:00.000',
 '00:59:24.000',
 '01:04:48.000',
 '01:10:12.000',
 '01:15:36.000',
 '01:21:00.000',
 '01:26:24.000',
 '01:31:48.000',
 '01:37:12.000',
 '01:42:36.000',
 '01:48:00.000',
 '01:53:24.000',
 '01:58:48.000',
 '02:04:12.000',
 '02:09:36.000',
 '02:15:00.000',
 '02:20:24.000',
 '02:25:48.000',
 '02:31:12.000',
 '02:36:36.000',
 '02:42:00.000',
 '02:47:24.000',
 '02:52:48.000',
 '02:58:12.000',
 '03:03:36.000',
 '03:09:00.000',
 '03:14:24.000',
 '03:19:48.000',
 '03:25:12.000',
 '03:30:36.000',
 '03:36:00.000',
 '03:41:24.000',
 '03:46:48.000',
 '03:52:12.000',
 '03:57:36.000',
 '04:03:00.000',
 '04:08:24.000',
 '04:13:48.000',
 '04:19:12.000',
 '04:24:36.000',
 '04:30:00.000',
 '04:35:24.000',
 '04:40:48.000',
 '04:46:12.000',
 '04:51:36.000',
 '04:57:00.000',
 '05:02:24.000',
 '05:07:48.000',
 '05:13:12.000

# Save json config files

In [53]:
k = 0
# generate 100 point in [0;36944] interval
for i in range(0, 36944, 370):
    datastore["generators"][1]["points"] = {}
    for idx, date_key in enumerate(dts):
        datastore["generators"][1]["points"][date_key] = all_ts_reshaped[i][idx]
    config_file_path = '/home/user/projects/tsimulus_ts_generate/generator_configuration_{}.json'.format(k+1)
    with open(config_file_path, 'w') as f:
        json.dump(datastore, f)
    save_path = "/home/user/time-series-compression/generated_datasets/generated_ts_{0}.txt".format(k+1)
    subprocess.call("java -jar /home/user/projects/tsimulus_ts_generate/tsimulus-cli.jar {0} > {1}".format(config_file_path, save_path), shell=True)
    k = k + 1

In [ ]:
         "points": {
            "00:00:00.000": -2.5,
            "01:00:00.000": 4.49,
            "02:00:00.000": 3.32,
            "03:00:00.000": 3.94,
            "04:00:00.000": 5.36,
            "05:00:00.000": 4.72,
            "06:00:00.000": 4.74,
            "07:00:00.000": 4.23,
            "08:00:00.000": 3.87,
            "09:00:00.000": 1.76,
            "10:00:00.000": 0.78,
            "11:00:00.000": -2.15,
            "12:00:00.000": -3.46,
            "13:00:00.000": -3.65,
            "14:00:00.000": -4.06,
            "15:00:00.000": -5.76,
            "16:00:00.000": -4.73,
            "17:00:00.000": -5.88,
            "18:00:00.000": -4.56,
            "19:00:00.000": -3.28,
            "20:00:00.000": -2.78,
            "21:00:00.000": -0.76,
            "22:00:00.000": 0.41,
            "23:00:00.000": 0.95
         }